In [ ]:
using PerlaTonettiWaugh, LinearAlgebra, Plots, BenchmarkTools

Set up parameters and find the corresponding stationary solution:

In [ ]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053) # Baselines per Jesse. 

# solve for stationary solution at t = 0
params_0 = merge(params, (d = d_0,)) # parameters to be used at t = 0
params_T = merge(params, (d = d_T,)) # parameters to be used at t = T

stationary_sol_0 = stationary_numerical(params_0, z_grid) # solution at t = 0
stationary_sol = stationary_numerical(params_T, z_grid) # solution at t = T

Ω_0 = stationary_sol_0.Ω
Ω_T = stationary_sol.Ω

println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

Choose `T` and `Ω_vec0` with its corresponding nodes `Ω_nodes`:

In [ ]:
Ω_NODES_COUNT = 5
α = 1.05
δ = params.δ - 0.0515
T = sqrt(2*(log(Ω_0) - log(Ω_T)) / ((2-α) * δ))
Ω(t) = Ω_0 * exp(-δ*T*t + α*δ*t*t/2) # Exponential Ω with time smoothing

In [ ]:
Ω_nodes = range(0.0, stop=T, length=Ω_NODES_COUNT)
Ω_nodes_interior = Ω_nodes[2:(end-1)]
Ω_vec0 = map(t -> Ω(t), Ω_nodes_interior) 

In [ ]:
settings = (z = z_grid, tstops = 0:1e-3:T, Δ_E = 1e-06)
entry_residuals_nodes = 1e-1:1e-1:(T-1e-1)

In [ ]:
# choose arbitrary Ω_vec0 if you wish
Ω_vec0[1] = 1.289
Ω_vec0[2] = 1.126
Ω_vec0[3] = 1.065

Return the entry residuals:

In [ ]:
# Solve and compute residuals, now using vectorized Ω
@time solved_entry_residuals = entry_residuals(Ω_vec0, Ω_0, stationary_sol, T, params_T, settings, Ω_nodes, entry_residuals_nodes);
Ω_interpolation = solved_entry_residuals.Ω_interpolation
entry_residuals_vec = solved_entry_residuals.entry_residuals
solved_tuple = solved_entry_residuals.solved_dynamics
solved = solved_tuple.results
v_t0 = solved_tuple.sol.u[1][1:M]
v0 = solved[:v_0]

# save v0 and v_hat_t0
v_hat_t0 = map(z -> exp((params.σ-1)*z), z_grid) .* v_t0;

## Plots for Ω and `entry_residuals`

In [ ]:
plot_Ω = plot(entry_residuals_nodes, Ω_interpolation.(entry_residuals_nodes), label = "Omega", lw = 3)
plot_residual = plot(entry_residuals_nodes, entry_residuals_vec, label = "entry_residual", lw = 3)
plot(plot_Ω, plot_residual, layout = (2,1))

In [ ]:
norm(entry_residuals_vec)

## Primary Plots

In [ ]:
plot1 = plot(solved.t, solved.g, label = "g", lw = 3)
plot2 = plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)
plot3 = plot(solved.t, solved.S, label = "S", lw = 3)
plot4 = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot1, plot2, plot3, plot4, layout=(2,2))

## Static Equations

In [ ]:
plot1 = plot(solved.t, solved.L_tilde, label = "L_tilde", lw = 3)
plot2 = plot(solved.t, solved.z_bar, label = "z_bar", lw = 3)
plot3 = plot(solved.t, solved.π_min, label = "pi_min", lw = 3)
plot4 = plot(solved.t, solved.λ_ii, label = "lambda_ii", lw = 3)
plot5 = plot(solved.t, solved.c, label = "c", lw = 3)
plot6 = plot(solved.t, solved.E, label = "E", lw = 3)
plot7 = plot(z_grid, v_hat_t0, label = "v_hat at t = 0", lw = 3)
plot(plot1, plot2, plot3, plot4, plot5, plot6, layout=(3,2))

In [ ]:
# Can examine the returned data with the Voyager and/or Vegalite
using DataVoyager, VegaLite
solved |> Voyager()
#solved |> @vlplot(:line, x = :t, y = :g, width=400, height=400)